In [1]:
import pulp
import cvxpy as cp
import random
import pandas as pd
from decimal import Decimal

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.DataFrame(
    [
        [10, 6, 16, 8],
        [8, 8, 4, 16],
        [12, 4, 8, ]
    ],
    columns=["店a", "店b", "店c", "供給量"],
    index=["工場x", "工場y", "需要量"]
)

df

,店a,店b,店c,供給量
工場x,10,6,16,8.0
工場y,8,8,4,16.0
需要量,12,4,8,NaN


## pulp

### 1.最適化問題の名称の設定と、最小化か最大化の設定

In [3]:
prob = pulp.LpProblem('sample', # 名称
                      pulp.LpMinimize) # 最小化。なおデフォルトは最小化

### 2.変数の設定

In [4]:
shop_list = ["a", "b", "c"]

In [5]:
# 工場Xの供給量
fact_x = [pulp.LpVariable(f'fact_x_{i}', 
                    lowBound = 0, # 下限値。デフォルト値はNone
                    upBound=None, # 上限値。デフォルト値はNone
                    cat=pulp.LpContinuous
                         # 変数の種類。デフォルトは"Continuous"(連続値)
                         # 他には pulp.LpInteger 整数、pulp.LpBinary バイナリ値
                    ) for i in shop_list]

# 工場yの供給量
fact_y = [pulp.LpVariable(f'fact_y_{i}', 
                    lowBound = 0, # 下限値。デフォルト値はNone
                    upBound=None, # 上限値。デフォルト値はNone
                    cat=pulp.LpContinuous
                         # 変数の種類。デフォルトは"Continuous"(連続値)
                         # 他には pulp.LpInteger 整数、pulp.LpBinary バイナリ値
                    ) for i in shop_list]

### 3.目的変数の設定

In [6]:
prob += pulp.lpSum(fact_x * df.iloc[0, :3]) + pulp.lpSum(fact_y * df.iloc[1, :3])

### 4.制約条件の設定

#### 需要量

In [7]:
for i in range(3):
    prob += fact_x[i] + fact_y[i] >= df.iloc[2, i]

#### 供給量

In [8]:
prob += pulp.lpSum(fact_x) <= df["供給量"][0]
prob += pulp.lpSum(fact_y) <= df["供給量"][1]

### 5.実行

In [9]:
status = prob.solve()
print("Status", pulp.LpStatus[status])

Status Optimal


In [10]:
result = pd.DataFrame(
    [
        [fact_x[i].value() for i in range(3)],
        [fact_y[i].value() for i in range(3)],
    ],
    columns=["店a", "店b", "店c"],
    index=["工場x", "工場y"]
)

result

,店a,店b,店c
工場x,4.0,4.0,0.0
工場y,8.0,0.0,8.0


In [11]:
prob.objective.value()

160.0

## cvxpy

### 1.変数の設定

In [13]:
fact_x = cp.Variable(
                    len(shop_list), # 要素数。2次元の場合は(x, y)と設定する
                    integer=False, # 整数の場合はTrueとする
                    boolean=False, # Binary値の場合はTrueとする
                    pos=True # 正の数
                    )

fact_y = cp.Variable(
                    len(shop_list), # 要素数。2次元の場合は(x, y)と設定する
                    integer=False, # 整数の場合はTrueとする
                     boolean=False, # Binary値の場合はTrueとする
                    pos=True # 正の数
                    )

### 2.目的変数の設定と最小化か最大化の設定

In [14]:
exp = cp.sum(fact_x * df.iloc[:1, :3].squeeze() + fact_y * df.iloc[1:2, :3].squeeze())

In [15]:
obj = cp.Minimize(exp)

### 3.制約条件の設定

#### 需要量

In [16]:
const = []

In [17]:
for i in range(3):
    const += [fact_x[i] + fact_y[i] >= df.iloc[2, i]]

#### 供給量

In [18]:
const += [cp.sum(fact_x) <= df["供給量"][0]]
const += [cp.sum(fact_y) <= df["供給量"][1]]

### 4.問題(目的変数・制約条件)の設定と実行

In [19]:
prob = cp.Problem(obj, const)
status = prob.solve(verbose=True)
status

                                     CVXPY                                     
                                    v1.1.11                                    
(CVXPY) Mar 29 12:11:16 AM: Your problem has 6 variables, 5 constraints, and 0 parameters.
(CVXPY) Mar 29 12:11:16 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 29 12:11:16 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 29 12:11:16 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 29 12:11:16 AM: Compiling problem (target solver=ECOS).
(CVXPY) Mar 29 12:11:16 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> 

159.9999999983278

In [22]:
pd.options.display.precision = 15

result = pd.DataFrame(
    [
        [fact_x[i].value for i in range(3)],
        [fact_y[i].value for i in range(3)],
    ],
    columns=["店a", "店b", "店c"],
    index=["工場x", "工場y"]
)

result

,店a,店b,店c
工場x,3.999999999942269,3.999999999887292,0.00000000000000
工場y,8.000000000041020,0.000000000018669,8.00000000000615


In [23]:
prob.value

159.9999999983278